<a href="https://colab.research.google.com/github/Paul9615/Paul_pytorch/blob/main/PART1/3_CNN/2_2_MNIST_ConvolutionNueralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms 
import torch.nn as nn
import torch.nn.init

In [3]:
# set GPU 
dev = 'cuda' if torch.cuda.is_available() else 'cpu'

# fix random seed 
torch.manual_seed(1)

# if can use GPU, fix the random seed 
if dev == 'cuda':
  torch.cuda.manual_seed_all(1)

print(dev)

cuda


In [5]:
# set parameters for use to training 
learn_rate = 0.001
n_epoch = 15
batch_size = 100

# define dataset 
train_data = dsets.MNIST(
    root = 'MNIST_data/',
    train = True,
    transform = transforms.ToTensor(),
    download=True
)

test_data = dsets.MNIST(
    root = 'MNIST_data/',
    train = False,
    transform = transforms.ToTensor(),
    download=True
)

# set batch size using dataloader 
data_loader = torch.utils.data.DataLoader(
    dataset = train_data,
    batch_size = batch_size,
    shuffle=True,
    drop_last = True
)

In [9]:
# design model 
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    # first layer 
    self.layer1 = nn.Sequential(
        nn.Conv2d(1,32,kernel_size=3, stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    
    # second layer 
    self.layer2 = nn.Sequential(
        nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    
    # fully connected layer 
    # 7x7x64 inputs -> 10 outputs
    self.fc = nn.Linear(7*7*64,10,bias=True)

    # initialize weighted in fully connected layer 
    nn.init.xavier_uniform_(self.fc.weight)

    # forward operation 
  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0),-1) # flatten for fully connected layer
    out = self.fc(out)
    return out

In [11]:
# define model
mod = CNN().to(dev)

# define cost function and optimizer 
criterion = nn.CrossEntropyLoss().to(dev)
opt = torch.optim.Adam(mod.parameters(), lr=learn_rate)

# total batch 
tot_batch = len(data_loader)
print(f'total batch: {tot_batch}')

total batch: 600


In [13]:
# trainint 
for epoch in range(n_epoch):
  avg_cost = 0
  for x, y in data_loader: # get data in mini batch, x is mini batch, y is label
    x = x.to(dev)
    y = y.to(dev)

    opt.zero_grad()
    hyp = mod(x)
    cost = criterion(hyp, y)
    cost.backward()
    opt.step()

    avg_cost += cost/tot_batch

  print(f'Epoch: {epoch+1:>4} Cost: {avg_cost:>.9}')

Epoch:    1 Cost: 0.00560098561
Epoch:    2 Cost: 0.00395665132
Epoch:    3 Cost: 0.00213228259
Epoch:    4 Cost: 0.00519693224
Epoch:    5 Cost: 0.00516313687
Epoch:    6 Cost: 0.00268676644
Epoch:    7 Cost: 0.0026588249
Epoch:    8 Cost: 0.00332005695
Epoch:    9 Cost: 0.00212878734
Epoch:   10 Cost: 0.00407589599
Epoch:   11 Cost: 0.00216609612
Epoch:   12 Cost: 0.000928715453
Epoch:   13 Cost: 0.00326016126
Epoch:   14 Cost: 0.000881520042
Epoch:   15 Cost: 0.00352526503


In [14]:
# Evaluate 
with torch.no_grad():
  x_test = test_data.test_data.view(len(test_data), 1,28,28).float().to(dev)
  y_test = test_data.test_labels.to(dev)

  pred = mod(x_test)
  corr_pred = torch.argmax(pred, 1) == y_test
  acc = corr_pred.float().mean()
  print(f'Accuracy: {acc.item()}')

Accuracy: 0.9801999926567078


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
